In [1]:
import os
import ase.io
from typing import Optional, List

import numpy as np
from matplotlib import pyplot as plt
import MDAnalysis as mda

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

/home/angiod@usi.ch/anaconda3/envs/fesnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root = "results/nmr-bmrb-temp-298-ph-6.5/debug"

In [12]:
### Load all test results from folder ###
#########################################

test_systems_data = []
for filename in os.listdir(root):
    if not filename.startswith("ds_"):
        continue
    if not filename.endswith("_target.xyz"):
        continue
    pred_frames = ase.io.read(os.path.join(root, filename[:-11] + ".xyz"), index=":", format="extxyz")
    target_frames = ase.io.read(os.path.join(root, filename), index=":", format="extxyz")
    name = filename.split('__')[1]
    system_data = []
    for pred, target in zip(pred_frames, target_frames):
        try:
            type = 'forces'
            frame_data = {
            'name': name,
            'coords': pred.arrays['positions'],
            'atom_types': pred.arrays['numbers'],
            'forces_pred': pred.arrays['forces'],
            'forces_target': target.arrays['forces'],
            'energies_pred': pred.info["energy"],
            'energies_target': target.info["energy"],
            }
        except KeyError:
            type = 'nmr'
            frame_data = {
            'name': name,
            'coords': pred.arrays['positions'],
            'atom_types': pred.arrays['numbers'],
            'energies_pred': pred.arrays["energies"],
            'energies_target': target.arrays["energies"],
            }
        system_data.append(frame_data)
    if len(system_data) == 0:
        frame_data = {
            'name': name,
        }
        system_data.append(frame_data)
    test_systems_data.append(system_data)

In [8]:
if type == 'forces':
    forces_means = []
    energies_means = []
    for data in test_systems_data[0]:
        forces_means.append(np.abs(data['forces_pred'] - data['forces_target']).mean())
        energies_means.append(np.abs(data['energies_pred'] - data['energies_target']).mean())
    forces_means = np.array(forces_means)
    energies_means = np.array(energies_means)

    fm = forces_means.mean()
    em = energies_means.mean()

    print(fm, em, fm * 43, em * 43) # kcal/mol | meV

In [9]:
if type == 'forces':
    x = []
    for data in test_systems_data[0]:
        x.append((data['forces_pred'] - data['forces_target']))
    x = np.array(x)

In [47]:
if type == 'nmr':
    evaluation_id = -1
    at_cs_means = {}
    for at in np.unique(test_systems_data[evaluation_id][0]['atom_types']):
        cs_means = []
        for data in test_systems_data[evaluation_id]:
            at_fltr = data['atom_types'] == at
            cs_means.append(np.nanmean(np.abs(data['energies_pred'][at_fltr] - data['energies_target'][at_fltr])))
        cs_means = np.array(cs_means)
        at_cs_means[at.item()] = cs_means

/tmp/ipykernel_3893835/3993900729.py:8: RuntimeWarning: Mean of empty slice
  cs_means.append(np.nanmean(np.abs(data['energies_pred'][at_fltr] - data['energies_target'][at_fltr])))


In [48]:
at_cs_means

{1: array([0.09339439, 0.11746112, 0.0910616 , 0.09211984, 0.1076199 ,
        0.09187434]),
 6: array([0.20412947, 0.20027313, 0.2346516 , 0.19601963, 0.18239738,
        0.27278088]),
 7: array([0.34060669, 0.29927698, 0.30400213, 0.26960246, 0.33724594,
        0.34420268]),
 8: array([nan, nan, nan, nan, nan, nan]),
 16: array([nan, nan, nan, nan, nan, nan])}